In [11]:
%matplotlib qt
import pyxdf
import mne
import numpy as np
from mne.datasets import misc
import os
import matplotlib.pyplot as plt
import matplotlib
import time
print(os.getcwd())

fnames = ["sub-ElpidaShield_ses-S001_task-DefaultASSR3_acq-UltracortexLayout1_run-001_eeg.xdf"]
raws = []
for fnameIndex in range(len(fnames)): 
    streams, header = pyxdf.load_xdf(fnames[fnameIndex])
    # get sampling rate
    sfreq = float(streams[1]["info"]["nominal_srate"][0])

    # get data
    data = streams[1]["time_series"].T
    dataTimestamps = streams[1]["time_stamps"].T
    # get markers
    markers = streams[0]["time_series"].T
    markerTimestamps = streams[0]["time_stamps"].T

    #in case of there is a switch between stim channel and data channels
    if  data.shape[0] != 8:
        data = streams[0]["time_series"].T
        dataTimestamps = streams[0]["time_stamps"].T
        sfreq = float(streams[0]["info"]["nominal_srate"][0])
        markers = streams[1]["time_series"].T
        markerTimestamps = streams[1]["time_stamps"].T
        assert data.shape[0] == 8,  fnames[fnameIndex] + " has different channel amounts" # check that we have what we think we have (8 eeg channels)

    

    # get together the event channel with the data, make a "stim 014" type channel as annotation: time, duration, event type 
    # add markers: What needs to be done: we have markers as several events, and their timestamp. then, we have the data
    # and their timestamps, to the raw object we should add one more channel, initially full of zeros with the length of the measurement
    
    
    # This is because microvolts are being measured, but mne plot expects values in volts.
    markers = markers[0]
    data *= 1e-6 # uV -> V and preamp gain. IDK what will preamp gain do. (multiply / divide by gain? 24?)

    markerArr = np.zeros_like(data[0])
    
    for markerid in range(len(markers)):
        absolute_differences = np.abs(dataTimestamps - markerTimestamps[markerid])
        closest_index = np.argmin(absolute_differences)
        markerArr[closest_index] = markers[markerid]

    # TEST: this gives the amount of events, and also their index. divided by the sampling freq, you should see the time differences of
    #each part of the experiment. 
    nonzero_indices = np.nonzero(markerArr)
    num_nonzero = len(nonzero_indices[0])
    print("uniques", np.unique(markerArr))
    print("Number of nonzero elements:", num_nonzero)
    print("Indices of nonzero elements:", nonzero_indices[0] / sfreq)

    #add marker arr to data
    combined = np.c_[data.T,markerArr].T

    # create info object set channel names
    info = mne.create_info(["T6", "T4", "F8", "C4", "C3", "T5", "T3", "F7", "stim"], sfreq, ["eeg", "eeg","eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "stim"])
    raw = mne.io.RawArray(combined, info)
    raw.set_montage("standard_1020")

    #preliminary data analyzis: (fancy term for plotting it and looking at it). Task: review data and select a consistently bad channel for exclusion
    print("Task: review data and select a consistently bad channel for exclusion.")
    
    raw.plot(title=fnames[fnameIndex] + ", Task: review data and select consistently bad channels", duration=1)
    raw.compute_psd().plot()
    plt.title(fnames[fnameIndex])
    plt.show()
    raws.append((raw, fnames[fnameIndex]))

c:\mindenJoSHasztalan\UniversityAndStuff\DTU\Semester 6\Thesis\External Software\MNE\Tutorial
uniques [0. 1. 2.]
Number of nonzero elements: 20
Indices of nonzero elements: [  4.748  14.668  74.84   84.752 144.924 154.84  215.004 224.92  285.084
 295.008 355.184 365.092 425.26  435.172 495.348 505.264 565.42  575.344
 635.508 645.416]
Creating RawArray with float64 data, n_channels=9, n_times=183600
    Range : 0 ... 183599 =      0.000 ...   734.396 secs
Ready.
Task: review data and select a consistently bad channel for exclusion.
Effective window size : 1.024 (s)


Channels marked as bad:
['T4', 'T3', 'T5']


In [18]:
epochs = []
for rawpack in raws: 
    raw = rawpack[0]
    fname = rawpack[1]
    low_cut = 0.1
    high_cut = 50
    filtered_data = raw.copy().filter(low_cut,high_cut,method='iir')
    notch_freqs = [50, 100]
    filtered_data = filtered_data.notch_filter(freqs=notch_freqs, filter_length='auto', method='fir', notch_widths=4, picks="eeg")

    #get events    
    events = mne.find_events(filtered_data, stim_channel="stim")
    #events: 1: start rest, 2: start assr, 3: artifact
    #setup values above which you reject (peak to peak)
    reject_criteria = dict(eeg=200e-6)

    #set flat criteria (epochs where nothing happens really (min peak to peak))
    flat_criteria = dict(eeg=1e-6)

    #make epochs
    stimEpochs = mne.Epochs(filtered_data, events, event_id=2, tmin=-10, tmax=60, baseline=(-10, 0), preload=True, reject=reject_criteria, flat=flat_criteria)
    stimEpochs.plot_drop_log()
    epochs.append((stimEpochs, fnames[fnameIndex]))

    stimEpochs.plot(title=fname+"filtered")
    stimEpochs.compute_psd().plot()
    plt.title(fname)
    plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 50.00 Hz: -6.02, -6.02 dB



Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 sec)

20 events found


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


Event IDs: [1 2]
Not setting metadata
10 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 10 events and 17501 original time points ...
    Rejecting  epoch based on EEG : ['T6', 'C4', 'C3', 'F7']
1 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Averaging across epochs...
